In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import datetime
import matplotlib.pyplot as plt

In [3]:
train_loader = torch.load("../data/train_loader.pt")
val_loader = torch.load("../data/val_loader.pt")
test_loader = torch.load("../data/test_loader.pt")

In [7]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.act1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(32, 16, kernel_size=3, padding=1)
        self.act2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.act3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(2)
        self.conv4 = nn.Conv2d(8, 4, kernel_size=3, padding=1)
        self.act4 = nn.ReLU()
        self.pool4 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(4 * 14 * 14, 32)
        self.act5 = nn.ReLU()
        self.fc2 = nn.Linear(32, 2)
        
    def forward(self, x):
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = self.pool3(self.act3(self.conv3(out)))
        out = self.pool4(self.act4(self.conv4(out)))
        out = out.view(-1, 4 * 14 * 14)
        out = self.act5(self.fc1(out))
        out = self.fc2(out)
        
        return out

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else torch.device("cpu"))

In [9]:
def training_loop(n_epochs, model, criterion, optimizer, train_loader, val_loader):
    train_losses = []
    val_losses = []
    for epoch in range(n_epochs):
        loss_train = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            loss_train += loss.item()
            
        avg_loss_train = loss_train / len(train_loader)
        train_losses.append(avg_loss_train)
        
        if val_loader is not None:
            loss_val = 0.0
            with torch.no_grad():
                for val_images, val_labels in val_loader:
                    val_images, val_labels = val_images.to(device), val_labels.to(device)
                    val_outputs = model(val_images)
                    loss_val += criterion(val_outputs, val_labels).item()
            avg_val_loss = loss_val / len(val_loader)
            val_losses.append(avg_val_loss)

        if epoch == 0 or (epoch + 1) % 10 == 0:
            print('{} Epoch {}, Training loss: {:.4f}'.format(
                datetime.datetime.now(), epoch + 1, avg_loss_train))
            if val_loader is not None:
                print('{} Epoch {}, Validation loss: {:.4f}'.format(
                    datetime.datetime.now(), epoch + 1, avg_val_loss))

    return train_losses, val_losses

In [ ]:
def plot_losses(train_losses, val_losses=None):
    plt.plot(train_losses, label='Training loss')
    if val_losses:
        plt.plot(val_losses, label='Validation loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Losses')
    plt.show()